In [1]:
%cd /home/ubuntu/shot-glass/python

import math
from importlib import reload

import lunchbox.tools as lbt
from lunchbox.enforce import Enforce, EnforceError
import pandas as pd
import numpy as np

import shot_glass.core.monad as sgm
from shot_glass.core.monad import *
from shot_glass.core.basic import *
# from shot_glass.core.df_monad import *
# from shot_glass.core.s_monad import *

/home/ubuntu/shot-glass/python


In [2]:
r = Try(1).fmap(lambda x: x + 1)                  
print('fmap', r.__class__.__name__, r)

r = Try(1).bind(lambda x: Monad(x + 1))           
print('bind', r.__class__.__name__, r)

r = Try(1).app(Monad(lambda x: x + 1))            
print('app', r.__class__.__name__, r)

r = Try('failsauce').fmap(lambda x: x + 1)        
print('fmap', r.__class__.__name__, r)

r = Try('failsauce').bind(lambda x: Monad(x + 1)) 
print('bind', r.__class__.__name__, r)

r = Try('failsauce').app(Monad(lambda x: x + 1))  
print('app', r.__class__.__name__, r)

fmap Try Success(2)
bind Monad Monad(2)
app Try Success(2)
fmap Try Failure(can only concatenate str (not "int") to str)
bind Try Failure(can only concatenate str (not "int") to str)
app Try Failure(can only concatenate str (not "int") to str)


In [3]:
Maybe(1).fmap(lambda x: x + 1)
# Maybe(1).fmap(lambda x: None)
# Maybe.nothing()

EnforceError: Just(1) is not a subclass or instance of Monad.

In [ ]:
class Maybe(Monad):
    @classmethod
    def just(cls, value):
        return cls(value)dev.l
    @classmethod
    def nothing(cls, value=None):
        return cls(value)

    def __repr__(self):
        if self.state == 'just':
            return f'Just({self._data})'
        return 'Nothing'
    
    @property
    def state(self):
        data = self._data
        if data is None or pd.isna(data):
            return 'nothing'
        return 'just'
    
    @classmethod
    def _try(cls, monad, method, func):
        if monad.state == 'nothing':
            return cls.nothing(monad.unwrap())
        return cls.just(monad.unwrap())

Maybe(1).fmap(lambda x: x + 1)
Maybe(1).fmap(lambda x: None)
# Maybe(1).fmap(lambda x: x + 'a')
m = Try(Maybe(1))
m.fmap(lambda x: x.unwrap() + 1)
m.fmap(lambda x: x.fmap(lambda x: x + 1))
m.fmap(lambda x: x.fmap(lambda x: None))
m.fmap(lambda x: x.fmap(lambda x: x + 'a'))

Failure(unsupported operand type(s) for +: 'int' and 'str')

In [128]:
data = pd.DataFrame()
data['foo'] = [None, 2, 3]
data['bar'] = [1, 'a', 'b']

In [129]:
m = DFMonad(data) \
    .wrap_elements(Maybe) \
    .bind_elements(Try.wrap)

# m = DFMonad(data) \
#     .wrap_elements(Try) \
#     .bind_elements(Maybe.wrap)
    
# m = DFMonad(data) \
    # .wrap_elements(Try) \
#     .bind_elements(Maybe)

m.unwrap().applymap(type)
# x = m.unwrap().loc[0, 'foo']
# x.unwrap()
# m
# m.fmap(type)
# x = m.unwrap().loc[0, 'foo']
# type(x), type(x.unwrap()), type(x.unwrap().unwrap())

,foo,bar
0,<class '__main__.Maybe'>,<class '__main__.Try'>
1,<class '__main__.Try'>,<class '__main__.Try'>
2,<class '__main__.Try'>,<class '__main__.Try'>


In [16]:
class Vertex(sgm.Monad):
    def __init__(self, data):
        super().__init__(data)
        self.validate()
    
    def validate(self):
        data = self._data
        assert len(data) == 3
        for i in data:
            assert type(i) in [int, float]

v = Vertex.wrap([2, 3, 1])
v

Vertex([2, 3, 1])